# Programación evolutiva aplicada a la función de Ackley

## Problema
Implementar un algoritmo basado en PE para resolver el siguiente problema:

$$
\text{min }f(\overrightarrow{x}) = - 20\;exp\left(-0.2\sqrt{\frac{1}{n}\sum_{i = 1}^{n}x_i^2}\right) - exp\left(\frac{1}{n}\sum_{i = 1}^{n}cos(2\pi x_i)\right) + 20 + e
$$

La función $f$ es conocida como la **función de Ackley**, su dominio es $|x_i|\leq 30$ y su mínimo global está en $x_i = 0$ y $f(\overrightarrow{x} = 0.0$

**Consideraciones**

* Cuidar que las variables de decisión no se salgan del rango especificado en el problema. Si eso ocurre, deberá contar con un mecanismo que vuelva a generar el valor de la variable o que lo ajuste al rango deseado.

La **entrada** al algoritmo será:
1. La primera línea tendrá el número de variables que tendrá la función de Ackley
2. La segunda línea tendrá el tamaño de la población y el número máximo de generaciones, separados por un espacio.
3. La tercera línea tendrá el valor de los parámetris $\alpha$ y $\epsilon_0$, separados por un espacio.

La **salida** del programa será:

1. La mejor solución encontrada.
2. El valor de la función objetivo para dicha solución.

A continuación se muestra un ejemplo:

**Entrada:**

2

100 200

2.0 0.0001

**Salida:**

[7.48188924e − 08, −6.67209514e − 07]

f(x) = 0.000

Permitiar al usuario realizar $M$ ejecuciones del algoritmos de PE, implementado en el punto anterior, para resolver una instancia del problema de Ackley. Después de las $M$ ejecuciones se debe reportar lo siguiente:

1. Mejor solución encontrada considerando las $M$ ejecuciones.
2. Peor solución encontrada considerando las $M$ ejecuciones.
3. Solución que corresponde a la mediana considerando las $M$ ejecuciones.
4. Media del valor de la función objetivo considerando las $M$ ejecuciones.
5. Desviación estándar del valor de la función objetivo considerando las $M$ ejecuciones.

En los primeros tres puntos indica tanto el valor de $x$ como el valor de la función objetivo $f$ (función de Ackley). *Nota*: Se deben reportar los resultados del algoritmo para las siguientes instancias: 5, 10 y 20 variables de decisión. Recuerde que puede variar los parámetros del algoritmo (tamaño de la población, número de generaciones, $\alpha$ y $\epsilon_0$). Indique los valores que se utilizaron para cada instancia del problema.

## Solución

In [30]:
import numpy as np

Comencemos por definir la función de Ackley cuyo parámetro a recibir es un arreglo de *numpy*.

In [31]:
def f(x):
    n = len(x)
    a1 = -20 * np.exp(-0.2 * np.sqrt((1/n) * sum(x**2)))
    a2 = -np.exp((1/n) * sum(np.cos(2*np.pi*x)))
    return a1 + a2 + 20 + np.e

Ya que tenemos esto, declaremos una estructura/clase que corresponda a un individuo. En este caso tendremos tres componentes por individuo:
* $\overrightarrow{x}$
* $\overrightarrow{\sigma}$
* $f(\overrightarrow{x})$

In [32]:
class Individual():
    def __init__(self, values = [], sigma = [], fVal = None):
        self.values = values
        self.sigma = sigma
        self.fVal = fVal
    
    def __repr__(self):
        return (f"Values = {self.values}\nSigma = {self.sigma}\nF(x) = {self.fVal}")
    

Una vez que contamos con esta clase, podemos empezar a definir los métodos que ocupamos. Recordemos que en PE, iniciamos con una población inicial de padres. Ésta la podemos generar de manera aleatoria de tal manera que $\overrightarrow{x}$ esté compuesto de valores uniformemente distribuidos entre 0 y 30; mientras que generemos $\overrightarrow{\sigma}$ con valores distribuidos de manera normal con media 0 y desviación estándar 1.

Esto se hace en la siguiente función, que recibe el número de individuos a generar y el número de variables de cada uno.

In [33]:
def initialPopulation(mu, n,): # mu = num. of individuals, n = num. of variables
    population = []
    
    #Generate mu individuals with n variables
    for i in range(mu):
        i = Individual()
        i.values = np.array([np.random.uniform(-30,30) for i in range(n)])
        i.sigma = np.array([np.random.normal(0,1) for i in range(n)])
        i.fVal = f(i.values)
        population.append(i)
        
    return population

Lo que sigue es tener un método de mutación de nuestra población. Cada miembro $i$ de la población genera un hijo $i'$ utilizando lo siguiente:

$$
\sigma_j'=\sigma_j\cdot(1+\alpha \cdot N(0,1))
$$

$$
x'_j = x_j + \sigma'_j \cdot N(0,1)
$$

Además, con este método, también debemos tener cuidado de no salirnos del rango $|x_i|\leq 30$. Esto lo vamos a evitar convirtiendo en $30$ todos aquellos $x_j^{(i)} > 30$ y en $-30$ todos aquellos $x_j^{(i)} < -30$ que aparezcan al hacer la mutación.


También, almacenaremos de una vez en cada hijo resultante el valor en $f(\overrightarrow{x})$.

Esto lo definimos en la siguiente celda:

In [34]:
def generateChildren(parents,e0,alpha):
    children = []
    
    for ind in parents:
        sigmap = [sj * (1+alpha*np.random.normal(0,1)) for sj in ind.sigma]
        sigmap = [e0 if abs(sj) < e0 else sj for sj in sigmap]
        
        xp = np.array([xj + sigmapj*np.random.normal(0,1) for xj,sigmapj in zip(ind.values,sigmap)])
        xp = np.array([30 if xpi > 30 else -30 if xpi < -30 else xpi for xpi in xp])
        
        child = Individual(values = xp, sigma = sigmap, fVal = f(xp))
        
        children.append(child)
        
        
    return children
        

Con estas dos funciones base, podemos implementar el algoritmo de PE. Primero generamos una población inicial de $\mu$ individuos con $n$ variables cada uno. Luego generamos los hijos de esta población por medio de una mutación para generar otros $\mu$ individuos. A partir de la unión entre los padres y los hijos nos quedamos únicamente con las mejores $\mu$ soluciones con base en su valor de $f$ y este nuevo conjunto es usado en la siguiente generación para producir más hijos y así sucesivamente. Esto se repite un total de $G$ generaciones.

Al final regresamos la mejor solución de todas las que hay en la última generación, tanto $\overrightarrow{x}$ como $f(\overrightarrow{x})$.

Definiéndolo tenemos:

In [35]:
"""
n = núm. variables
mu = tamaño poblacion
G = num. generaciones
e0 = menor sd permitida
alpha = param. mutacion
"""

def PE(n, mu, G, alpha, e0): #
    parents = initialPopulation(mu,n)
    for i in range(G):
        children = generateChildren(parents, e0, alpha)
        
        parents += children
        parents.sort(key = lambda x : x.fVal)
        parents = parents[:mu]

    return parents[0].values, parents[0].fVal

Ya tenemos todo lo necesario para ejecutar el algoritmo de PE. Como ejemplo, usemos el mismo que viene en el ejemplo de entrada y salida. Para leer los datos de un archivo de texto con datos como el del ejemplo usemos la siguiente función.

In [36]:
def readInput(fileName):
  with open(fileName) as f:
      lines = f.readlines()
      n = int(lines[0])
      mu, G = map(int, lines[1].split(" "))
      alpha, e0 = map(float, lines[2].split())
    
  return n, mu, G, alpha, e0

In [38]:
n,mu,G,alpha,e0 = readInput("data.txt")

Dándonos los siguientes resultados

In [41]:
x,fx = PE(n,mu,G,alpha,e0)

print(x,"\nf(x) =",fx)

[-4.06933345e-07  1.07690512e-06] 
f(x) = 3.2561917717721656e-06


Ya que hicimos una prueba inicial del algoritmo, podemos ver cómo la solución del mínimo de la función de Ackley sí se aproximam mucho al resultado que buscamos ($f(x) = 0$) cuando trabajamos con dos variables.

Lo que sigue es probar esto para $n = 2,5,10,20$. En este caso vamos a intentar usar $\alpha = 0.2$, ya que es de los recomendados. Vamos a usar una población inicial de $\mu = 300$ y además generaremos $G = 600$ generaciones. El valor de $\epsilon_0$ lo mantendremos como 0.0001


In [54]:
mu = 300
G = 600
alpha = 0.2
epsilon = 0.0001

Esto lo haremos para $n = 2,5,10,20$, que son los valores que pide el problema.

#### Caso $n = 2$

In [56]:
x,fx = PE(2,mu,G,alpha,epsilon)
print(x)
print(f"f(x) = {fx}")

[-7.90390593e-08  4.60859678e-08]
f(x) = 2.5878336318285733e-07


#### Caso $n = 5$

In [57]:
x,fx = PE(5,mu,G,alpha,epsilon)
print(x)
print(f"f(x) = {fx}")

[-1.68883913e-06 -1.75779131e-05 -2.72103055e-06  9.66393614e-06
  1.30798265e-05]
f(x) = 4.3225203413488344e-05


#### Caso $n = 10$

In [59]:
x,fx = PE(10,mu,G,alpha,epsilon)
print(x)
print(f"f(x) = {fx}")

[-8.36991897e-06 -3.63011890e-05 -7.89410927e-05  3.87718539e-05
 -3.15641227e-05  7.82286298e-06 -2.99781642e-05 -4.77074664e-05
 -1.11645132e-05  6.96748595e-06]
f(x) = 0.00014719463790813236


#### Caso $n = 20$

In [60]:
x,fx = PE(20,mu,G,alpha,epsilon)
print(x)
print(f"f(x) = {fx}")

[-3.74263347e-04 -1.59657361e+01  2.99413641e+00 -9.97796542e-01
 -9.97443071e-01  2.99264772e+00 -1.99516494e+00  1.79614861e+01
 -9.98010746e-01  4.98901440e+00 -9.98394037e-01 -3.51270185e-04
 -3.46087972e-04 -9.97813592e-01 -1.99570261e+00 -2.99299647e+00
  9.97810205e-01 -9.97040361e-01  3.73147672e-04  9.97351599e-01]
f(x) = 13.585808110784308


Notemos, cómo conforme aumentaba el valor de $n$, más empeoraba el resultado del método. Esto implica que mientras más variables ocupamos, más ajustes tenemos que hacer, probablemente en el número de individuos, $\mu$ y de generaciones, $G$.

Ahora continuemos con la última parte del problema: generar un reporte tras $M$ ejecuciones. Definamos una función que se encarge de eso.

In [86]:
def generateReport(mu,G,alpha,epsilon):
  print("M = ", end = "")
  M = int(input())
  print("N = " , end = "")
  N = int(input())

  results = []
  for i in range(M):
    results.append(PE(N,mu,G,alpha,epsilon))

  results = sorted(results, key = lambda x:x[1])
  print(f"1. Best answer = \n{results[0][0]}\nf(x) = {results[0][1]}\n")
  print(f"2. Worst answer = \n{results[M-1][0]}\nf(x) = {results[M-1][1]}\n")
  print(f"3. Median answer = \n{results[(M-1)//2][0]}\nf(x) = {results[(M-1)//2][1]}\n")
  res = np.array([r[1] for r in results])
  print(f"4. f(x) mean = {np.mean(res)}")
  print(f"5. f(x) standard deviation = {np.std(res)}")

Generemos 4 distintos reporte con $M = 50$ $\mu = 100$, $G = 200$, $\alpha = 2$, $\epsilon_0 = 0.0001$. De momento elegir, valores más grandes para $\mu$, $M$ o $G$ causaría tiempos de ejecución bastante largos. Por ello, solo nos enfocaremos en demostrar en que esto sirva para el número de variables de $n = 2,5,10,20$ con valores no tan altos.

#### Reporte para $n = 2$

In [88]:
generateReport(100,200,2,0.0001)

M = 50
N = 2
1. Best answer = 
[-5.39239162e-07  2.37780179e-07]
f(x) = 1.6669064533125777e-06

2. Worst answer = 
[ 1.06442695e-06 -4.81891887e-06]
f(x) = 1.3959153851050843e-05

3. Median answer = 
[ 1.53906261e-06 -1.25736364e-06]
f(x) = 5.621263085675565e-06

4. f(x) mean = 6.054789098630664e-06
5. f(x) standard deviation = 2.925603552878764e-06


#### Reporte para $n = 5$



In [89]:
generateReport(100,200,2,0.0001)

M = 50
N = 5
1. Best answer = 
[-7.59894287e-05 -2.07514618e-05  5.86896248e-05  3.52005709e-05
 -1.23842388e-05]
f(x) = 0.00018809155065158123

2. Worst answer = 
[ 2.96726298e-04  3.90295364e-04  1.91884251e-04  6.73778007e-05
 -2.91301175e-04]
f(x) = 0.0010870005252674453

3. Median answer = 
[-1.26782521e-04 -2.28966066e-05 -4.93048398e-05 -8.50551849e-05
  1.00688216e-04]
f(x) = 0.0003416865660779145

4. f(x) mean = 0.0004136229353313503
5. f(x) standard deviation = 0.00019494107441061603


#### Reporte para $n = 10$


In [90]:
generateReport(100,200,2,0.0001)

M = 50
N = 10
1. Best answer = 
[-0.02556044  0.12792732  0.00070081  0.02693412 -0.00134194 -0.02378064
 -0.01105714  0.02556138 -0.05685636  0.09629911]
f(x) = 0.3819543113152162

2. Worst answer = 
[ 6.39703379e-02 -1.57955406e-01 -6.74632481e-02 -1.26158293e-01
 -7.76339193e-01  2.19121538e-01  7.23919013e-04  9.67011144e-01
  2.36808781e-01  9.46306958e-02]
f(x) = 2.418707809849455

3. Median answer = 
[-0.08911854 -0.01155726  0.13129284  0.01047047  0.08960055  0.13777702
  0.09892789  0.08562779  0.0594949   0.97364293]
f(x) = 1.5923998039801508

4. f(x) mean = 1.5209187692979527
5. f(x) standard deviation = 0.6026108445960925


#### Reporte para $n = 20$


In [91]:
generateReport(100,200,2,0.0001)

M = 50
N = 20
1. Best answer = 
[ 0.84433605 -0.64437657 -1.11765598 -0.75965053  3.20673174 -1.89646933
 -2.71933339  1.22711958  4.05084863 -2.43851518  0.077588   -2.21212055
 -4.70273781 -2.46375855 -0.70246948 -2.76634458 -2.03800757  2.58642224
 -2.1879462   1.10691218]
f(x) = 8.957244727665747

2. Worst answer = 
[ 5.15075243  1.32865662  2.31607232  1.86203111 -0.2891711   0.93857196
 -6.80515319  3.03501652 -0.27836424 -8.23604648 -4.54315731 -3.04454764
 -5.15387818  2.03534746 -2.26875677  3.49255122  1.87389294  1.67975985
  1.97448697 24.15829623]
f(x) = 15.966432669142238

3. Median answer = 
[ 0.63952362  3.94688983 -5.23619035  0.05948577  6.98300563  3.2130264
 -0.59784137 -0.92462342 -3.38812682  0.07355187 -2.08827849 -5.17221943
 10.10333434 -5.10298576 -0.67102741 -7.24721642 -0.21535152 -1.40621058
  1.37094754  0.18446571]
f(x) = 12.616908090999752

4. f(x) mean = 12.756225441661636
5. f(x) standard deviation = 1.6836262020401462


Tras observar estos reportes, podemos observar claramente cómo conforme aumenta el número de variables a evaluar, peores resultados tenemos; además de que el tiempo de cómputo aumenta significativamente. Para mejorar los resultados ocupamos ajustar valores de los parámetros como en incrementos para $\mu$ y $G$, lo cual implica mucho tiempo de ejecución del algoritmo.

A pesar de todo ello, es más que evidente que este algoritmo con los parámetros adecuados es muy bueno escapando de mínimos locales. Esto se demuestra al circular por una función llena de varios puntos mínimos como la de Ackley, donde alcanzamos en varios de los casos al mínimo global.